<a href="https://colab.research.google.com/github/1Un/Dota2/blob/main/hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!unzip '/content/gdrive/MyDrive/Colab Notebooks/Coursera/hw7/data.zip' -d '/content/'
!unzip '/content/gdrive/MyDrive/Colab Notebooks/Coursera/hw7/features.zip' -d '/content/features'
!unzip '/content/gdrive/MyDrive/Colab Notebooks/Coursera/hw7/features_test.zip' -d '/content/features_test'

Mounted at /content/gdrive
Archive:  /content/gdrive/MyDrive/Colab Notebooks/Coursera/hw7/data.zip
   creating: /content/data/
   creating: /content/data/dictionaries/
  inflating: /content/data/dictionaries/abilities.csv  
  inflating: /content/data/dictionaries/heroes.csv  
  inflating: /content/data/dictionaries/items.csv  
  inflating: /content/data/dictionaries/lobbies.csv  
  inflating: /content/data/dictionaries/mods.csv  
  inflating: /content/data/dictionaries/regions.csv  
  inflating: /content/data/extract_features.py  
  inflating: /content/data/features.csv  
  inflating: /content/data/features_test.csv  
  inflating: /content/data/matches.jsonlines.bz2  
 extracting: /content/data/matches_test.jsonlines.bz2  
  inflating: /content/data/sample_submission.csv  
Archive:  /content/gdrive/MyDrive/Colab Notebooks/Coursera/hw7/features.zip
  inflating: /content/features/features.csv  
Archive:  /content/gdrive/MyDrive/Colab Notebooks/Coursera/hw7/features_test.zip
  inflating: 

In [2]:
import pandas as pd
import numpy as np

# Подход 1


## №1 
Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в 
описании данных как отсутствующие в тестовой выборке).

In [3]:
features_train = pd.read_csv('/content/features/features.csv', index_col='match_id')
features_test=pd.read_csv('/content/features_test/features_test.csv',index_col='match_id')


In [4]:
features_train=features_train.drop(['barracks_status_dire',
                                    'barracks_status_radiant',
                                    'duration',
                                    'radiant_win',
                                    'tower_status_dire',
                                    'tower_status_radiant'],
                                    axis=1)

## №2
Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [5]:
s=(features_train.shape[0]-features_train.count())
s.loc[s>0]


first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64

In [6]:
s=(features_test.shape[0]-features_test.count())
s.loc[s>0]


first_blood_time               3552
first_blood_team               3552
first_blood_player1            3552
first_blood_player2            7766
radiant_bottle_time            2895
radiant_courier_time            127
radiant_flying_courier_time    4885
radiant_first_ward_time         330
dire_bottle_time               2842
dire_courier_time               130
dire_flying_courier_time       4524
dire_first_ward_time            263
dtype: int64

first_blood_team: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
В данном столбце пропущенные значние могут показывать 

In [7]:
features_train.loc[features_train.first_blood_team.isna()==True,['first_blood_team','r1_kills','r2_kills']].head()
# Если посмотреть на данные,то мы увидим что r1_kills=0 и r1_kills=0 т.е. в игре не было смертей =>
# не было первого убийства, т.е. там и стоит nan, хотя можно наверное было ввести напр. -1(т.е.
# это обозначала что first blodd отсутсвует). В остальных столбцах тоже видна такая закономерность что,
# если в игре что тоне случилось, напр.что nun в столбце dire_bottle_time означает что, во время игры не был приобретён botl, 


,first_blood_team,r1_kills,r2_kills
match_id,,,
3,NaN,0,0
9,NaN,0,0
45,NaN,0,0
51,NaN,0,0
86,NaN,0,0


## №3
Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [8]:
features_train=features_train.fillna(value=0)
features_test=features_test.fillna(value=0)

## №4
Какой столбец содержит целевую переменную? Запишите его название.

In [9]:
#radiant_win
y=pd.read_csv('/content/features/features.csv', index_col='match_id')[['radiant_win']]
y.head()

,radiant_win
match_id,
0,1
1,1
2,0
3,0
4,0


## №5

Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


In [11]:
clf=GradientBoostingClassifier()
kf=KFold(n_splits=5,shuffle=True,random_state=42)
params={'n_estimators':[10,20,30],
        'learning_rate':[1],
        'random_state':[42]}

In [12]:
gs=GridSearchCV(estimator=clf,param_grid=params,cv=kf,verbose=True,n_jobs=-1,scoring='roc_auc')

In [13]:
import time
import datetime
 
start_time = datetime.datetime.now()
 
gs.fit(features_train,y.radiant_win)
 
print(f'Time elapsed:{datetime.datetime.now() - start_time}')



Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  4.1min finished


Time elapsed:0:04:46.422804


In [14]:
gs.best_params_

{'learning_rate': 1, 'n_estimators': 30, 'random_state': 42}

In [15]:
gs.best_score_ 

0.696238940721182

In [16]:
gs.cv_results_

{'mean_fit_time': array([15.74466147, 31.7975419 , 45.41812496]),
 'mean_score_time': array([0.04959512, 0.05223961, 0.05666389]),
 'mean_test_score': array([0.68394606, 0.69324505, 0.69623894]),
 'param_learning_rate': masked_array(data=[1, 1, 1],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 20, 30],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_random_state': masked_array(data=[42, 42, 42],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'learning_rate': 1, 'n_estimators': 10, 'random_state': 42},
  {'learning_rate': 1, 'n_estimators': 20, 'random_state': 42},
  {'learning_rate': 1, 'n_estimators': 30, 'random_state': 42}],
 'rank_test_score': array([3, 2, 1], dtype=int32),
 'split0_test_score': array([0.68250838, 0.69009126, 0.69427617]),
 'split1_test_score': array([0.6844786

In [17]:
#видим что  при увеличении числа деревьев скор улучшается только в 3 знаке после
#запятой, так что резона увеличить число n_estimators нету

# Подход 2

## №1 
Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [19]:
scaler = StandardScaler()
X_train=scaler.fit_transform(features_train)
X_test=scaler.fit_transform(features_test)

In [20]:
clf_reg=LogisticRegression()
param_reg={'penalty':['l2'],
           'C':[1,0.5,0.1,0.05,0.01,0.005]}

gs_reg=GridSearchCV(estimator=clf_reg,param_grid=param_reg,cv=kf,verbose=True,n_jobs=-1,scoring='roc_auc')

In [21]:
start_time = datetime.datetime.now()
 
gs_reg.fit(X_train,y.radiant_win)
 
print(f'Time elapsed:{datetime.datetime.now() - start_time}')


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   28.5s finished


Time elapsed:0:00:30.027159


In [22]:
gs_reg.best_params_

{'C': 0.005, 'penalty': 'l2'}

In [23]:
gs_reg.cv_results_['std_test_score']

array([0.00123566, 0.00123512, 0.00123585, 0.00123649, 0.00123805,
       0.0012379 ])

In [24]:
gs_reg.best_score_

0.7165585432045204

## №2
Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [25]:
cols = ['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']


In [26]:
indx=[features_train.columns.get_loc(indx) for indx in cols]
indx

[1, 2, 10, 18, 26, 34, 42, 50, 58, 66, 74]

In [27]:
gs_reg.best_estimator_.coef_[0][indx]
#После удаления категориальных признаков качество не изменило. 
#Это могло произошло потому что веса при этих параметрах были близки к нулю.

array([-0.00525989,  0.00550341, -0.01963529,  0.00208191,  0.00242044,
       -0.01117869,  0.00011369,  0.00268805, -0.00344395,  0.00647685,
        0.0042156 ])

In [28]:
features_train=features_train.drop(cols,axis=1)
features_test=features_test.drop(cols,axis=1)


In [29]:
X_train=scaler.fit_transform(features_train)
X_test=scaler.fit_transform(features_test)

In [30]:
start_time = datetime.datetime.now()
 
gs_reg.fit(X_train,y.radiant_win)
 
print(f'Time elapsed:{datetime.datetime.now() - start_time}')


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   26.6s finished


Time elapsed:0:00:28.313749


In [31]:
gs_reg.best_params_

{'C': 0.005, 'penalty': 'l2'}

In [32]:
gs_reg.cv_results_['std_test_score']

array([0.00132269, 0.00132322, 0.00132319, 0.00132373, 0.00132875,
       0.00133124])

In [33]:
gs_reg.best_score_

0.7165646342612926

## №3
На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [34]:
heroes = pd.read_csv('/content/data/dictionaries/heroes.csv',index_col='id')

In [35]:
heroes.shape[0]

112

## №4
Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [55]:
features_train = pd.read_csv('/content/features/features.csv', index_col='match_id').fillna(0)
features_train.drop(['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire','radiant_win']
           , axis=1, inplace=True)
features_test=pd.read_csv('/content/features_test/features_test.csv',index_col='match_id').fillna(0)



In [56]:
X_pick = np.zeros((features_train.shape[0], heroes.shape[0]))
for i, match_id in enumerate(features_train.index):
    for p in range(5):
        X_pick[i, features_train.loc[match_id, 'r%d_hero' % (p + 1)] - 1] = 1
        X_pick[i, features_train.loc[match_id, 'd%d_hero' % (p + 1)] - 1] = -1

In [57]:
X_hero = pd.DataFrame(X_pick, index=features_train.index)
features_train = pd.concat([features_train, X_hero], axis=1)

In [58]:
X_pick = np.zeros((features_test.shape[0], heroes.shape[0]))
for i, match_id in enumerate(features_test.index):
    for p in range(5):
        X_pick[i, features_test.loc[match_id, 'r%d_hero' % (p + 1)] - 1] = 1
        X_pick[i, features_test.loc[match_id, 'd%d_hero' % (p + 1)] - 1] = -1

X_hero = pd.DataFrame(X_pick, index=features_test.index)
features_test = pd.concat([features_test, X_hero], axis=1)

In [59]:
X_train=scaler.fit_transform(features_train)
X_test=scaler.fit_transform(features_test)

## №5 
Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [41]:
start_time = datetime.datetime.now()
 
gs_reg.fit(X_train,y.radiant_win)
 
print(f'Time elapsed:{datetime.datetime.now() - start_time}')


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.0min finished


Time elapsed:0:01:04.834694


In [42]:
gs_reg.best_params_

{'C': 0.005, 'penalty': 'l2'}

In [43]:
gs_reg.best_score_
#Добавление "мешка слов" улучшило качество предсказания. 

0.7519225640157142

In [62]:
#Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
y_pred = gs_reg.predict_proba(X_test)[:, 1]

In [63]:
print(f'\tминимальная - {round(y_pred.min(), 3)}')
print(f'\tмаксимальная - {round(y_pred.max(), 3)}')

	минимальная - 0.009
	максимальная - 0.996
